In [ ]:
!pip install streamlit --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.7 MB/s eta 0:00:00


In [ ]:
%%writefile Ddos_Attack_Detection2.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Title of the Streamlit app
st.title("Data Understanding and Exploration")

# Upload CSV file
uploaded_file = st.file_uploader("Upload CSV file", type=["csv"])
if uploaded_file is not None:
    # Read the data from the CSV file in chunks to handle large files
    chunk_size = 10000
    df = pd.concat([chunk for chunk in pd.read_csv(uploaded_file, chunksize=chunk_size)])

    # Initial Data Exploration
    st.header("Initial Data Exploration")
    st.write("Shape of the DataFrame:")
    st.write(df.shape)

    # Adjust display settings
    pd.set_option('display.max_columns', 100)
    pd.set_option('display.max_rows', 100)
    pd.set_option('display.width', 1000)
    pd.set_option('display.max_colwidth', 20)

    st.header("Descriptive Statistics")
    st.write(df.describe())

    # Frequency Counts for Categorical Variable
    if ' Label' in df.columns:
        st.header("Frequency Counts for 'Label'")
        frequency_counts = df[' Label'].value_counts()
        st.write(frequency_counts)

        label_counts = df[' Label'].value_counts()
        colors = plt.cm.viridis(np.linspace(0, 1, len(label_counts)))

        plt.figure(figsize=(12, 8))
        bars = plt.bar(label_counts.index, label_counts.values, color=colors, edgecolor='black')
        plt.grid(True, which='major', linestyle='--', linewidth='0.5', color='grey')
        plt.gca().set_axisbelow(True)

        for i, val in enumerate(label_counts):
            plt.text(i, val, str(val), ha='center', va='bottom')

        plt.title('Bar Chart of Label')
        plt.xlabel(' Label')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45, ha='right')
        plt.legend(bars, label_counts.index, title="Types", bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        st.pyplot(plt.gcf())

    # Data Preparation
    st.header("Data Preparation")

    # Drop Irrelevant Features
    st.subheader("Drop Irrelevant Features")
    initial_columns = df.columns.tolist()
    irrelevant_features = ['Unnamed: 0', 'Flow ID', ' Source IP', ' Source Port', ' Destination IP', ' Destination Port', ' Timestamp', ' Protocol', 'SimillarHTTP', ' Inbound']
    df = df.drop(irrelevant_features, axis=1)
    dropped_features = [feature for feature in irrelevant_features if feature in initial_columns]
    st.write(f"Features dropped: {dropped_features}")
    st.write("DataFrame after dropping irrelevant features:")
    st.write(df.head())

    additional_features_to_drop = [' Init_Win_bytes_backward', 'Init_Win_bytes_forward', ' Fwd Header Length.1', 'Fwd IAT Total']
    df = df.drop(columns=additional_features_to_drop, errors='ignore')
    dropped_additional_features = [feature for feature in additional_features_to_drop if feature in initial_columns]
    st.write(f"Additional features dropped: {dropped_additional_features}")
    st.write("DataFrame and Shape after dropping additional specified columns:")
    st.write(df.head())
    st.write(df.shape)

    # Drop Duplicate Rows
    duplicate_rows = df.duplicated()
    st.write("Number of duplicate rows: ", duplicate_rows.sum())
    df.drop_duplicates(inplace=True)
    st.write("DataFrame and Shape after dropping duplicate rows:")
    st.write(df.head())
    st.write(df.shape)

    # Drop Missing Values (Rows)
    missing_values = df.isnull().sum()
    st.write("Missing values in each column before cleaning:\n", missing_values)
    df = df.dropna()
    missing_values_cleaned = df.isnull().sum()
    st.write("Missing values in each column after cleaning:\n", missing_values_cleaned)
    st.write(df.shape)

    # Identify and Replace Infinite Values
    inf_values = df.select_dtypes(include=[np.number]).applymap(np.isinf)
    inf_counts = inf_values.sum()
    st.write("Infinite values in each column:\n", inf_counts)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    st.write("Infinite values after replacing infinite values and dropping NaNs:")
    st.write(df.shape)

    # Drop Columns with More Than 50% Zero Values
    zero_val_percent = (df == 0).astype(int).sum(axis=0) / df.shape[0] * 100
    threshold = 50
    columns_to_drop = zero_val_percent[zero_val_percent > threshold].index
    st.write(f"Columns identified for dropping (greater than {threshold}% zeros):")
    st.write(columns_to_drop)
    df.drop(columns=columns_to_drop, inplace=True)
    st.write(df.shape)

    # Drop Columns with More Than 50% Negative Values
    negative_columns = [col for col in df.select_dtypes(include=['int64', 'float64']).columns if (df[col] < 0).mean() > 0.5]
    st.write("Columns with more than 50% negative values:")
    st.write(negative_columns)
    df = df.drop(columns=negative_columns)
    st.write("Shape after dropping columns with more than 50% negative values: ", df.shape)

    # Drop Rows with Any Negative Value
    numerical_columns = df.select_dtypes(include=['int64', 'float64'])
    negative_values = numerical_columns < 0
    records_with_negative = df[negative_values.any(axis=1)]
    st.write("Number of rows with negative values in numerical columns:", len(records_with_negative))
    columns_with_negative = numerical_columns.columns[(negative_values.any())]
    st.write("Numerical columns with negative values:")
    st.write(columns_with_negative)
    df = df[~negative_values.any(axis=1)]
    st.write("Shape after dropping rows with negative values: ", df.shape)

    # Frequency Counts for Categorical Variable
    st.header("Frequency Counts for 'Label'")
    frequency_counts = df[' Label'].value_counts()
    st.write(frequency_counts)

    label_counts = df[' Label'].value_counts()
    colors = plt.cm.viridis(np.linspace(0, 1, len(label_counts)))

    plt.figure(figsize=(12, 8))
    bars = plt.bar(label_counts.index, label_counts.values, color=colors, edgecolor='black')
    plt.grid(True, which='major', linestyle='--', linewidth='0.5', color='grey')
    plt.gca().set_axisbelow(True)

    for i, val in enumerate(label_counts):
        plt.text(i, val, str(val), ha='center', va='bottom')

    plt.title('Bar Chart of Label')
    plt.xlabel(' Label')
    plt.ylabel('Frequency ')
    plt.xticks(rotation=45, ha='right')
    plt.legend(bars, label_counts.index, title="Types", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    st.pyplot(plt.gcf())

    # Data Transformation - Label Encoder
    st.header("Data Transformation - Label Encoder")
    df[' Label'] = df[' Label'].apply(lambda x: 0 if x == 'BENIGN' else 1)
    st.write("Label distribution after transformation:")
    st.write(df[' Label'].value_counts())

    plt.figure(figsize=(8, 6))
    ax = sns.countplot(x=' Label', data=df, palette='coolwarm')
    plt.title('Distribution of Labels')
    plt.xlabel(' Label')
    plt.ylabel('Count')
    plt.xticks([0, 1], ['Benign (0)', 'DDoS (1)'])
    for p in ax.patches:
        ax.annotate(f'{int(p.get_height()):,}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 10), textcoords='offset points')
    st.pyplot(plt)

    # Data Standardization
    st.header("Data Standardization")
    X = df.drop(' Label', axis=1)
    y = df[' Label']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    df_scaled = pd.DataFrame(X_scaled, columns=X.columns)
    df_scaled[' Label'] = y.values

    st.write("Standardized DataFrame")
    st.write(df_scaled.head())

    # Data Balancing
    st.header("Data Balancing")
    df_majority = df_scaled[df_scaled[' Label'] == 0]
    df_minority = df_scaled[df_scaled[' Label'] == 1]

    desired_sample_size = 7000

    df_majority_undersampled = df_majority.sample(n=desired_sample_size, random_state=42)
    df_minority_undersampled = df_minority.sample(n=desired_sample_size, random_state=42)

    df_balanced = pd.concat([df_majority_undersampled, df_minority_undersampled])

    plt.figure(figsize=(10, 6))
    ax = sns.countplot(x=' Label', data=df_balanced, palette='pastel')
    plt.title('Distribution of Labels (Balanced)')
    for p in ax.patches:
        ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='baseline', fontsize=12, color='black', xytext=(0, 5),
                    textcoords='offset points')
    plt.xlabel('Label')
    plt.ylabel('Count')
    plt.xticks([0, 1], ['Benign (0)', 'DDoS (1)'])
    st.pyplot(plt)

    # Save the DataFrame to a CSV file
    df_balanced.to_csv('Cleaned_Dataset.csv', index=False)
    st.write("File saved as 'Cleaned_Dataset.csv'")

    # Load and Read Dataset (CIC-DDoS2019 - Day 2)
    st.header("Load and Read Dataset (CIC-DDoS2019 - Day 2)")
    df_cleaned = pd.read_csv('Cleaned_Dataset.csv')
    st.write("First few rows of the cleaned dataset:")
    st.write(df_cleaned.head())


# Data Splitting (70:30)
    st.header("Data Splitting (60:40)")
    X = df_cleaned.drop(' Label', axis=1)
    y = df_cleaned[' Label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
    st.write("Training set (X):", X_train.shape)
    st.write("Training set (y):", y_train.shape)
    st.write("Test set (X):", X_test.shape)
    st.write("Test set (y):", y_test.shape)

    # Plot the distribution of the target in the training and testing sets
    plt.figure(figsize=(14, 7))

    plt.subplot(1, 2, 1)
    train_counts = y_train.value_counts()
    train_bar = train_counts.plot(kind='bar', title='Distribution of target in training set', color='skyblue')
    plt.xticks(rotation=0)
    for i, value in enumerate(train_counts):
        plt.text(i, value + value * 0.01, f'{value}', ha='center', va='bottom')
        plt.xticks([0, 1], ['Benign (0)', 'DDoS (1)'])

    plt.subplot(1, 2, 2)
    test_counts = y_test.value_counts()
    test_bar = test_counts.plot(kind='bar', title='Distribution of target in testing set', color='salmon')
    plt.xticks(rotation=0)
    for i, value in enumerate(test_counts):
        plt.text(i, value + value * 0.01, f'{value}', ha='center', va='bottom')
        plt.xticks([0, 1], ['Benign (0)', 'DDoS (1)'])

    plt.subplots_adjust(wspace=0.3)
    plt.suptitle('Target Distribution in Training and Testing Sets in CIC-DDoS2019')
    train_bar.grid(axis='y', linestyle='--', alpha=0.7)
    test_bar.grid(axis='y', linestyle='--', alpha=0.7)
    plt.subplot(1, 2, 1).set_facecolor('whitesmoke')
    plt.subplot(1, 2, 2).set_facecolor('whitesmoke')
    st.pyplot(plt)

    # Feature Selection PCA (60:40)
    st.header("Feature Selection PCA (60:40)")
    pca = PCA(n_components=25)
    pca.fit(X_train)
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    st.write("Training set after PCA (X):", X_train_pca.shape)
    st.write("Test set after PCA (X):", X_test_pca.shape)

    feature_names = X_train.columns.tolist()
    pca_components = pd.DataFrame(pca.components_, columns=feature_names)

    pc1_contributions = pca_components.iloc[0]
    pc2_contributions = pca_components.iloc[1]

    plt.figure(figsize=(14, 6))
    plt.bar(feature_names, pc1_contributions, alpha=0.5, label='PC1')
    plt.bar(feature_names, pc2_contributions, alpha=0.5, label='PC2', bottom=pc1_contributions)
    plt.title('Feature Contributions to the First Two Principal Components')
    plt.xlabel('Feature Names')
    plt.ylabel('Contribution')
    plt.xticks(rotation=90)
    plt.legend()
    st.pyplot(plt)

    # Feature Selection LDA (60:40)
    st.header("Feature Selection LDA (60:40)")
    lda = LDA(n_components=1)
    lda.fit(X_train, y_train)
    X_train_lda = lda.transform(X_train)
    X_test_lda = lda.transform(X_test)
    st.write("Training set after LDA (X):", X_train_lda.shape)
    st.write("Test set after LDA (X):", X_test_lda.shape)

    lda_coefficients = lda.coef_[0]
    plt.figure(figsize=(12, 6))
    plt.bar(feature_names, lda_coefficients)
    plt.xlabel('Feature Names')
    plt.ylabel('LDA Coefficient')
    plt.title('LDA Coefficients for Each Feature')
    plt.xticks(rotation=90)
    st.pyplot(plt)


    #### Default Parameter [kernel = linear] - TRAIN - WITHOUT FS

    # Initialize an SVM with a linear kernel
    svm_train_linear_wf = SVC(kernel="linear")

    # Measure training time
    start_time = time.time()
    svm_train_linear_wf.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_linear_wf = svm_train_linear_wf.predict(X_train)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_linear_wf)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_linear_wf)

    # Display the results in Streamlit
    st.header("SVM (Kernel: Linear) - Without Feature Selection - Default Parameters")
    st.subheader("Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Default Parameter [kernel = linear] - TEST

    # Measure testing time
    start_time = time.time()
    svm_train_linear_wf.fit(X_test, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_linear_wf = svm_train_linear_wf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_linear_wf)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_linear_wf)

    # Display the results in Streamlit
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ### Implementation of SVM (Kernel-Linear) -Train and Test Set - Without Feature Selection. - HALVING GRID SEARCH

    # Define the parameter grid
    param_grid = {'C': [1, 10]}

    # Initialize the SVM model
    svm = SVC()

    # Initialize HalvingGridSearchCV for faster results
    halving_grid_search = HalvingGridSearchCV(svm, param_grid, cv=5, factor=2, n_jobs=-1)
    halving_grid_search.fit(X_train, y_train)

    best_params = halving_grid_search.best_params_
    st.subheader("Best Hyperparameters Tuning")
    st.write(f"Best hyperparameters for Kernel Linear without Feature Selection : {best_params}")

    #### Tuning Parameter [kernel: linear, C=10] - TRAIN

    # Initialize an SVM with a linear kernel
    svm_train_linear_wf = SVC(kernel="linear", C=10)

    # Measure training time
    start_time = time.time()
    svm_train_linear_wf.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_linear_wf = svm_train_linear_wf.predict(X_train)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_linear_wf)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_linear_wf)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Kernel: Linear, C=10) - Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Tuning Parameter [kernel: linear, C=10] - TEST

    # Measure testing time
    start_time = time.time()
    svm_train_linear_wf.fit(X_test, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_linear_wf = svm_train_linear_wf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_linear_wf)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_linear_wf)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Kernel: Linear, C=10) - Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)


    #### Default Parameter [kernel = linear] - TRAIN - WITH PCA

    # Initialize an SVM with a linear kernel
    svm_train_linear_f = SVC(kernel="linear")

    # Measure training time
    start_time = time.time()
    svm_train_linear_f.fit(X_train_pca, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_linear_f = svm_train_linear_f.predict(X_train_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_linear_f)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_linear_f)

    # Display the results in Streamlit
    st.header("SVM (Kernel: Linear) - With PCA - Default Parameters")
    st.subheader("Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Default Parameter [kernel = linear] - TEST

    # Measure testing time
    start_time = time.time()
    svm_train_linear_f.fit(X_test_pca, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_linear_f = svm_train_linear_f.predict(X_test_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_linear_f)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_linear_f)

    # Display the results in Streamlit
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ### Implementation of SVM (Kernel-Linear) -Train and Test Set - Without Feature Selection. - HALVING GRID SEARCH

    # Define the parameter grid
    param_grid = {'C': [1, 10]}

    # Initialize the SVM model
    svm = SVC()

    # Initialize HalvingGridSearchCV for faster results
    halving_grid_search = HalvingGridSearchCV(svm, param_grid, cv=5, factor=2, n_jobs=-1)
    halving_grid_search.fit(X_train_pca, y_train)

    best_params = halving_grid_search.best_params_
    st.subheader("Best Hyperparameters Tuning")
    st.write(f"Best hyperparameters for Kernel Linear with PCA : {best_params}")

    #### Tuning Parameter [kernel: linear, C=10] - TRAIN

    # Initialize an SVM with a linear kernel
    svm_train_linear_f = SVC(kernel="linear", C=10)

    # Measure training time
    start_time = time.time()
    svm_train_linear_f.fit(X_train_pca, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_linear_f = svm_train_linear_f.predict(X_train_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_linear_f)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_linear_f)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Kernel: Linear, C=10) - Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Tuning Parameter [kernel: linear, C=10] - TEST

    # Measure testing time
    start_time = time.time()
    svm_train_linear_f.fit(X_test_pca, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_linear_f = svm_train_linear_f.predict(X_test_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_linear_f)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_linear_f)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Kernel: Linear, C=10) - Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)


    ## Implementation : Support Vector Machine KERNEL: POLYNOMIAL

    ### Implementation of SVM (Kernel- Polynomial) Train and Test Set - Without Feature Selection - DEFAULT

    #### Default Parameter [kernel="poly"] - TRAIN

    # Initialize an SVM with a poly kernel
    svm_train_poly_wf = SVC(kernel="poly")

    # Measure training time
    start_time = time.time()
    svm_train_poly_wf.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_poly_wf = svm_train_poly_wf.predict(X_train)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_poly_wf)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_poly_wf)

    # Display the results in Streamlit
    st.header("SVM (Kernel: Polynomial) - Without Feature Selection - Default Parameters")
    st.subheader("Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Default Parameter [kernel="poly"] - TEST

    # Measure testing time
    start_time = time.time()
    svm_train_poly_wf.fit(X_test, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_poly_wf = svm_train_poly_wf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_poly_wf)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_poly_wf)

    # Display the results in Streamlit
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ### Implementation of SVM (Kernel-Poly) -Train and Test Set - Without Feature Selection. - HALVING GRID SEARCH

    # Define the parameter grid
    param_grid = {'C': [1, 10], 'degree': [2, 3]}

    # Initialize the SVM model
    svm = SVC()

    # Initialize HalvingGridSearchCV for faster results
    halving_grid_search = HalvingGridSearchCV(svm, param_grid, cv=5, factor=2, n_jobs=-1)
    halving_grid_search.fit(X_train, y_train)

    best_params = halving_grid_search.best_params_
    st.subheader("Best Hyperparameters Tuning")
    st.write(f"Best hyperparameters for Kernel Polynomial without Feature Selection: {best_params}")

    #### Tuning Parameter [kernel: poly, C=10, degree=3] - TRAIN

    # Initialize an SVM with a poly kernel
    svm_train_poly_wf = SVC(kernel="poly", C=10, degree=3)

    # Measure training time
    start_time = time.time()
    svm_train_poly_wf.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_poly_wf = svm_train_poly_wf.predict(X_train)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_poly_wf)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_poly_wf)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Kernel: Polynomial, C=10, Degree=3) - Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Tuning Parameter [kernel: poly, C=10, degree=3] - TEST

    # Measure testing time
    start_time = time.time()
    svm_train_poly_wf.fit(X_test, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_poly_wf = svm_train_poly_wf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_poly_wf)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_poly_wf)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Kernel: Polynomial, C=10, Degree=3) - Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ### Implementation of SVM (Kernel-Polynomial) -Train and Test Set - With PCA. - DEFAULT

    #### Default Parameter [kernel="poly"] - TRAIN

    # Train the SVM classifier with a polynomial kernel on the selected features
    svm_train_poly_f = SVC(kernel="poly")

    # Measure training time
    start_time = time.time()
    svm_train_poly_f.fit(X_train_pca, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_poly_f = svm_train_poly_f.predict(X_train_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_poly_f)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_poly_f)

    # Display the results in Streamlit
    st.header("SVM (Kernel: Polynomial) - With PCA - Default Parameters")
    st.subheader("Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Default Parameter [kernel="poly"] - TEST

    # Measure testing time
    start_time = time.time()
    svm_train_poly_f.fit(X_test_pca, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_poly_f = svm_train_poly_f.predict(X_test_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_poly_f)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_poly_f)

    # Display the results in Streamlit
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ### Implementation of SVM (Kernel-Poly) -Train and Test Set - With PCA - HALVING GRID SEARCH

    # Define the parameter grid
    param_grid = {'C': [1, 10], 'degree': [2, 3]}

    # Initialize the SVM model
    svm = SVC()

    # Initialize HalvingGridSearchCV for faster results
    halving_grid_search = HalvingGridSearchCV(svm, param_grid, cv=5, factor=2, n_jobs=-1)
    halving_grid_search.fit(X_train_pca, y_train)

    best_params = halving_grid_search.best_params_
    st.subheader("Best Hyperparameters Tuning")
    st.write(f"Best hyperparameters for Kernel Polynomial with PCA: {best_params}")

    #### Tuning Parameter [kernel: poly, C=10, degree=3] - TRAIN

    # Train the SVM classifier with a polynomial kernel on the selected features
    svm_train_poly_f = SVC(kernel="poly", degree=3, C=10)

    # Measure training time
    start_time = time.time()
    svm_train_poly_f.fit(X_train_pca, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_poly_f = svm_train_poly_f.predict(X_train_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_poly_f)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_poly_f)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Kernel: Polynomial, C=10, Degree=3) - Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Tuning Parameter [kernel: poly, C=10, degree=3] - TEST

    # Measure testing time
    start_time = time.time()
    svm_train_poly_f.fit(X_test_pca, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_poly_f = svm_train_poly_f.predict(X_test_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_poly_f)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_poly_f)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Kernel: Polynomial, C=10, Degree=3) - Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ## Implementation : Support Vector Machine KERNEL: RBF

    ### Implementation of SVM (Kernel - RBF) - Train and Test Set - Without Feature Selection - DEFAULT

    #### Default Parameter [kernel = rbf] - TRAIN

    # Initialize an SVM with a rbf kernel
    svm_train_rbf_wf = SVC(kernel="rbf")

    # Measure training time
    start_time = time.time()
    svm_train_rbf_wf.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_rbf_wf = svm_train_rbf_wf.predict(X_train)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_rbf_wf)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_rbf_wf)

    # Display the results in Streamlit
    st.header("SVM (Kernel: RBF) - Without Feature Selection - Default Parameters")
    st.subheader("Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Default Parameter [kernel = rbf] - TEST

    # Measure testing time
    start_time = time.time()
    svm_train_rbf_wf.fit(X_test, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_rbf_wf = svm_train_rbf_wf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_rbf_wf)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_rbf_wf)

    # Display the results in Streamlit
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ### Implementation of SVM (Kernel RBF) - Train and Test Set - Without Feature Selection - HALVING GRID SEARCH

    # Define the parameter grid
    param_grid = {'C': [1, 10], 'gamma': ['scale', 'auto']}

    # Initialize the SVM model
    svm = SVC()

    # Initialize HalvingGridSearchCV for faster results
    halving_grid_search = HalvingGridSearchCV(svm, param_grid, cv=5, factor=2, n_jobs=-1)
    halving_grid_search.fit(X_train, y_train)

    best_params = halving_grid_search.best_params_
    st.subheader("Best Hyperparameters Tuning")
    st.write(f"Best hyperparameters for Kernel RBF without Feature Selection: {best_params}")

    #### Tuning Parameter [kernel: rbf, C=10, gamma=scale] - TRAIN

    # Initialize an SVM with a rbf kernel
    svm_train_rbf_wf = SVC(kernel="rbf", C=10, gamma='scale')

    # Measure training time
    start_time = time.time()
    svm_train_rbf_wf.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_rbf_wf = svm_train_rbf_wf.predict(X_train)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_rbf_wf)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_rbf_wf)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Kernel: RBF, C=10, Gamma=scale) - Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Tuning Parameter [kernel: rbf, C=10, gamma=scale] - TEST

    # Measure testing time
    start_time = time.time()
    svm_train_rbf_wf.fit(X_test, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_rbf_wf = svm_train_rbf_wf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_rbf_wf)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_rbf_wf)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Kernel: RBF, C=10, Gamma=scale) - Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ### Implementation of SVM (Kernel-RBF) using PCA - Train and Test set - DEFAULT

    #### Default Parameter [kernel=rbf] - TRAIN

    # Initialize an SVM with a rbf kernel
    svm_train_rbf_f = SVC(kernel="rbf")

    # Measure training time
    start_time = time.time()
    svm_train_rbf_f.fit(X_train_pca, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_rbf_f = svm_train_rbf_f.predict(X_train_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_rbf_f)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_rbf_f)

    # Display the results in Streamlit
    st.header("SVM (Kernel: RBF) - With PCA - Default Parameters")
    st.subheader("Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Default Parameter [kernel=rbf] - TEST

    # Measure testing time
    start_time = time.time()
    svm_train_rbf_f.fit(X_test_pca, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_rbf_f = svm_train_rbf_f.predict(X_test_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_rbf_f)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_rbf_f)

    # Display the results in Streamlit
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ### Implementation of SVM (Kernel-RBF) using PCA - Train and Test set - HALVING GRID SEARCH

    # Define the parameter grid
    param_grid = {'C': [1, 10], 'gamma': ['scale', 'auto']}

    # Initialize the SVM model
    svm = SVC()

    # Initialize HalvingGridSearchCV for faster results
    halving_grid_search = HalvingGridSearchCV(svm, param_grid, cv=5, factor=2, n_jobs=-1)
    halving_grid_search.fit(X_train_pca, y_train)

    best_params = halving_grid_search.best_params_
    st.subheader("Best Hyperparameters Tuning")
    st.write(f"Best hyperparameters for Kernel RBF using PCA: {best_params}")

    #### Tuning Parameter [kernel=rbf, C=10, gamma=auto] - TRAIN

    # Initialize an SVM with a rbf kernel
    svm_train_rbf_f = SVC(kernel="rbf", C=10, gamma='auto')

    # Measure training time
    start_time = time.time()
    svm_train_rbf_f.fit(X_train_pca, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_rbf_f = svm_train_rbf_f.predict(X_train_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_rbf_f)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_rbf_f)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Kernel: RBF, C=10, Gamma=auto) - Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Tuning Parameter [kernel=rbf, C=10, gamma=auto] - TEST

    # Measure testing time
    start_time = time.time()
    svm_train_rbf_f.fit(X_test_pca, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_rbf_f = svm_train_rbf_f.predict(X_test_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_rbf_f)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_rbf_f)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Kernel: RBF, C=10, Gamma=auto) - Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)




    # KNN (70:30)

    ## Implementation for KNN (Euclidean)

    ### Implementation of KNN (Euclidean) -- Train and Test Set  Without Feature Selection.- DEFAULT

    #### Default Parameter [metric=Euclidean] -TRAIN

    # Initialize an knn with a euclidean Metric
    knn_train_euclidean_wf = KNeighborsClassifier(n_neighbors=91, metric='euclidean', p=2) #sqrt

    # Measure training time
    start_time = time.time()
    knn_train_euclidean_wf.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_euclidean_wf = knn_train_euclidean_wf.predict(X_train)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_euclidean_wf)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_euclidean_wf)

    # Display the results in Streamlit
    st.header("KNN (Euclidean) - Without Feature Selection - Default Parameters")
    st.subheader("Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Default Parameter [metric=Euclidean] -TEST

    # Initialize an knn with a euclidean Metric
    knn_test_euclidean_wf = KNeighborsClassifier(n_neighbors=91,metric='euclidean', p=2)

    start_time = time.time()
    knn_test_euclidean_wf.fit(X_test, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_euclidean_wf = knn_test_euclidean_wf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_euclidean_wf)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_euclidean_wf)

    # Display the results in Streamlit
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ### Implementation of KNN (Euclidean) -- Train and Test Set  Without Feature Selection.- HALVING GRID SEARCH

    from sklearn.experimental import enable_halving_search_cv
    from sklearn.model_selection import HalvingGridSearchCV

    # Define the parameter grid
    param_grid = {
        'n_neighbors': [87,89],
        'metric': ['euclidean']
    }

    # Initialize the KNN model
    knn = KNeighborsClassifier()

    # Initialize HalvingGridSearchCV for faster results
    halving_grid_search = HalvingGridSearchCV(knn, param_grid, cv=5, factor=2,  n_jobs=-1)

    halving_grid_search.fit(X_train, y_train)

    best_params = halving_grid_search.best_params_
    st.subheader("Best Hyperparameters Tuning")
    st.write(f"Best hyperparameters: {best_params}")

    ####  Tuning Parameter [metric=Euclidean,'n_neighbors': 87] -TRAIN

    # Initialize an knn with a euclidean Metric
    knn_train_euclidean_wf = KNeighborsClassifier(n_neighbors=87, metric='euclidean', p=2)

    # Measure training time
    start_time = time.time()
    knn_train_euclidean_wf.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_euclidean_wf = knn_train_euclidean_wf.predict(X_train)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_euclidean_wf)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_euclidean_wf)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Euclidean, n_neighbors=87) - Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Tuning Parameter [metric=Euclidean'n_neighbors': 87] -TEST

    # Initialize an knn with a euclidean Metric
    knn_test_euclidean_wf = KNeighborsClassifier(n_neighbors=87,metric='euclidean', p=2)

    start_time = time.time()
    knn_test_euclidean_wf.fit(X_test, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_euclidean_wf = knn_test_euclidean_wf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_euclidean_wf)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_euclidean_wf)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Euclidean, n_neighbors=87) - Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ### Implementation of KNN (Metric-euclidean) -Train Set and Test Set - With PCA. - DEFAULT

    #### Default Parameter [metric=Euclidean] -TRAIN

    # Initialize an knn with a euclidean Metric
    knn_train_euclidean_f = KNeighborsClassifier(n_neighbors=91,metric='euclidean', p=2) #sqrt

    # Measure training time
    start_time = time.time()
    knn_train_euclidean_f.fit(X_train_pca, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_euclidean_f = knn_train_euclidean_f.predict(X_train_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_euclidean_f)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_euclidean_f)

    # Display the results in Streamlit
    st.header("KNN (Euclidean) - With PCA - Default Parameters")
    st.subheader("Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Default Parameter [metric=Euclidean] -TEST

    # Initialize an knn with a euclidean Metric
    knn_test_euclidean_f = KNeighborsClassifier(n_neighbors=91,metric='euclidean', p=2) #sqrt

    # Measure testing time
    start_time = time.time()
    knn_test_euclidean_f.fit(X_test_pca, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_euclidean_f = knn_test_euclidean_f.predict(X_test_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_euclidean_f)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_euclidean_f)

    # Display the results in Streamlit
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ### Implementation of KNN (Euclidean) -- Train and Test Set  With PCA.- HALVING GRID SEARCH

    # Define the parameter grid
    param_grid = {
        'n_neighbors': [87,89],
        'metric': ['euclidean']
    }

    # Initialize the KNN model
    knn = KNeighborsClassifier()

    # Initialize HalvingGridSearchCV for faster results
    halving_grid_search = HalvingGridSearchCV(knn, param_grid, cv=5, factor=2,  n_jobs=-1)

    halving_grid_search.fit(X_train_pca, y_train)

    best_params = halving_grid_search.best_params_
    st.subheader("Best Hyperparameters Tuning")
    st.write(f"Best hyperparameters: {best_params}")

    ####  Tuning Parameter [metric=Euclidean,'n_neighbors': 87] -TRAIN

    # Initialize an knn with a euclidean Metric
    knn_train_euclidean_f = KNeighborsClassifier(n_neighbors=87,metric='euclidean', p=2) #sqrt

    # Measure training time
    start_time = time.time()
    knn_train_euclidean_f.fit(X_train_pca, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_euclidean_f = knn_train_euclidean_f.predict(X_train_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_euclidean_f)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_euclidean_f)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Euclidean, n_neighbors=87) - Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Tuning Parameter [metric=Euclidean'n_neighbors': 95] -TEST

    # Initialize an knn with a euclidean Metric
    knn_test_euclidean_f = KNeighborsClassifier(n_neighbors=87,metric='euclidean', p=2) #sqrt

    # Measure testing time
    start_time = time.time()
    knn_test_euclidean_f.fit(X_test_pca, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_euclidean_f = knn_test_euclidean_f.predict(X_test_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_euclidean_f)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_euclidean_f)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Euclidean, n_neighbors=87) - Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)



    ## Implementation for KNN (Manhattan)

    ### Implementation of KNN (Manhattan) -- Train and Test Set  Without Feature Selection.- DEFAULT

    #### Default Parameter [metric-Manhattan] -TRAIN

    # Initialize an knn with a manhattan Metric
    knn_train_manhattan_wf = KNeighborsClassifier(n_neighbors=91, metric='manhattan', p=1) #sqrt

    # Measure training time
    start_time = time.time()
    knn_train_manhattan_wf.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_manhattan_wf = knn_train_manhattan_wf.predict(X_train)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_manhattan_wf)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_manhattan_wf)

    # Display the results in Streamlit
    st.header("KNN (Manhattan) - Without Feature Selection - Default Parameters")
    st.subheader("Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Default Parameter [metric=Manhattan] -TEST

    # Initialize an knn with a manhattan Metric
    knn_test_manhattan_wf = KNeighborsClassifier(n_neighbors=91, metric='manhattan', p=1)

    start_time = time.time()
    knn_test_manhattan_wf.fit(X_test, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_manhattan_wf = knn_test_manhattan_wf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_manhattan_wf)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_manhattan_wf)

    # Display the results in Streamlit
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)


    ### Implementation of KNN (Manhattan) -- Train and Test Set  Without Feature Selection.- HALVING GRID SEARCH

    # Define the parameter grid
    param_grid = {
        'n_neighbors': [87,89],
        'metric': ['manhattan']
    }

    # Initialize the KNN model
    knn = KNeighborsClassifier()

    # Initialize HalvingGridSearchCV for faster results
    halving_grid_search = HalvingGridSearchCV(knn, param_grid, cv=5, factor=2,  n_jobs=-1)

    halving_grid_search.fit(X_train, y_train)

    best_params = halving_grid_search.best_params_
    st.subheader("Best Hyperparameters Tuning")
    st.write(f"Best hyperparameters: {best_params}")

    ####  Tuning Parameter [metric=Manhattan,'n_neighbors': 87] -TRAIN

    # Initialize an knn with a manhattan Metric
    knn_train_manhattan_wf = KNeighborsClassifier(n_neighbors=87, metric='manhattan', p=1)

    # Measure training time
    start_time = time.time()
    knn_train_manhattan_wf.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_manhattan_wf = knn_train_manhattan_wf.predict(X_train)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_manhattan_wf)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_manhattan_wf)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Manhattan, n_neighbors=87) - Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ####  Tuning Parameter [metric=Manhattan,'n_neighbors': 87] -TEST

    # Initialize an knn with a manhattan Metric
    knn_test_manhattan_wf = KNeighborsClassifier(n_neighbors=87, metric='manhattan', p=1)

    start_time = time.time()
    knn_test_manhattan_wf.fit(X_test, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_manhattan_wf = knn_test_manhattan_wf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_manhattan_wf)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_manhattan_wf)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Manhattan, n_neighbors=87) - Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)


    ### Implementation of knn (Metric-manhattan) -Train Set and Test Set - With PCA. - DEFAULT

    #### Default Parameter [metric-Manhattan] -TRAIN

    # Initialize an knn with a manhattan Metric
    knn_train_manhattan_f = KNeighborsClassifier(n_neighbors=91, metric='manhattan', p=1) #sqrt

    # Measure training time
    start_time = time.time()
    knn_train_manhattan_f.fit(X_train_pca, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_manhattan_f = knn_train_manhattan_f.predict(X_train_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_manhattan_f)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_manhattan_f)

    # Display the results in Streamlit
    st.header("KNN (Manhattan) - With PCA - Default Parameters")
    st.subheader("Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Default Parameter [metric-Manhattan] -TEST

    # Initialize an knn with a manhattan Metric
    knn_test_manhattan_f = KNeighborsClassifier(n_neighbors=91, metric='manhattan', p=1) #sqrt

    # Measure testing time
    start_time = time.time()
    knn_test_manhattan_f.fit(X_test_pca, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_manhattan_f = knn_test_manhattan_f.predict(X_test_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_manhattan_f)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_manhattan_f)

    # Display the results in Streamlit
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)


    ### Implementation of KNN (Manhattan) -- Train and Test Set  With PCA.- HALVING GRID SEARCH

    # Define the parameter grid
    param_grid = {
        'n_neighbors': [87,89],
        'metric': ['manhattan']
    }

    # Initialize the KNN model
    knn = KNeighborsClassifier()

    # Initialize HalvingGridSearchCV for faster results
    halving_grid_search = HalvingGridSearchCV(knn, param_grid, cv=5, factor=2,  n_jobs=-1)

    halving_grid_search.fit(X_train_pca, y_train)

    best_params = halving_grid_search.best_params_
    st.subheader("Best Hyperparameters Tuning")
    st.write(f"Best hyperparameters: {best_params}")

    ####  Tuning Parameter [metric=Manhattan,'n_neighbors': 87] -TRAIN

    # Initialize an knn with a manhattan Metric
    knn_train_manhattan_f = KNeighborsClassifier(n_neighbors=87, metric='manhattan', p=1) #sqrt

    # Measure training time
    start_time = time.time()
    knn_train_manhattan_f.fit(X_train_pca, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_manhattan_f = knn_train_manhattan_f.predict(X_train_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_manhattan_f)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_manhattan_f)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Manhattan, n_neighbors=87) - Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ####  Tuning Parameter [metric=Manhattan,'n_neighbors': 87] -TEST

    # Initialize an knn with a manhattan Metric
    knn_test_manhattan_f = KNeighborsClassifier(n_neighbors=87, metric='manhattan', p=1) #sqrt

    # Measure testing time
    start_time = time.time()
    knn_test_manhattan_f.fit(X_test_pca, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_manhattan_f = knn_test_manhattan_f.predict(X_test_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_manhattan_f)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_manhattan_f)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Manhattan, n_neighbors=87) - Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)


    ## Implementation for KNN (Cosine)

    ### Implementation of KNN (cosine) -Train Set & Test Set - Without Feature Selection. - Default

    #### Default Parameter [metric=Cosine] -TRAIN

    # Initialize an knn with a cosine Metric
    knn_train_cosine_wf = KNeighborsClassifier(n_neighbors=91, metric='cosine') #sqrt

    # Measure training time
    start_time = time.time()
    knn_train_cosine_wf.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_cosine_wf = knn_train_cosine_wf.predict(X_train)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_cosine_wf)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_cosine_wf)

    # Display the results in Streamlit
    st.header("KNN (Cosine) - Without Feature Selection - Default Parameters")
    st.subheader("Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Default Parameter [metric=Cosine] -TEST

    # Initialize an knn with a cosine Metric
    knn_test_cosine_wf = KNeighborsClassifier(n_neighbors=91, metric='cosine')

    start_time = time.time()
    knn_test_cosine_wf.fit(X_test, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_cosine_wf = knn_test_cosine_wf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_cosine_wf)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_cosine_wf)

    # Display the results in Streamlit
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ### Implementation of KNN (cosine) -Train Set & Test Set - Without Feature Selection. - HALVING GRID SEARCH

    # Define the parameter grid
    param_grid = {
        'n_neighbors': [87,89],
        'metric': ['cosine']
    }

    # Initialize the KNN model
    knn = KNeighborsClassifier()

    # Initialize HalvingGridSearchCV for faster results
    halving_grid_search = HalvingGridSearchCV(knn, param_grid, cv=5, factor=2,  n_jobs=-1)

    halving_grid_search.fit(X_train, y_train)

    best_params = halving_grid_search.best_params_
    st.subheader("Best Hyperparameters Tuning")
    st.write(f"Best hyperparameters: {best_params}")

    #### Tuning Parameter [metric=Cosine, 'n_neighbors': 87] -TRAIN

    # Initialize an knn with a cosine Metric
    knn_train_cosine_wf = KNeighborsClassifier(n_neighbors=87, metric='cosine') #sqrt

    # Measure training time
    start_time = time.time()
    knn_train_cosine_wf.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_cosine_wf = knn_train_cosine_wf.predict(X_train)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_cosine_wf)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_cosine_wf)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Cosine, n_neighbors=87) - Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Tuning Parameter [metric=Cosine,'n_neighbors': 87] -TEST

    # Initialize an knn with a cosine Metric
    knn_test_cosine_wf = KNeighborsClassifier(n_neighbors=87, metric='cosine')

    start_time = time.time()
    knn_test_cosine_wf.fit(X_test, y_test)
    testing_time = time.time() - start_time

   # Measure prediction time
    start_time = time.time()
    y_test_pred_cosine_wf = knn_test_cosine_wf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_cosine_wf)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_cosine_wf)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Cosine, n_neighbors=87) - Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    ### Implementation of KNN (cosine) -Train Set & Test Set -With PCA - Default

    #### Default Parameter [metric=Cosine] -TRAIN

    # Initialize an knn with a cosine Metric
    knn_train_cosine_f = KNeighborsClassifier(n_neighbors=99, metric='cosine') #sqrt

    # Measure training time
    start_time = time.time()
    knn_train_cosine_f.fit(X_train_pca, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_cosine_f = knn_train_cosine_f.predict(X_train_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_cosine_f)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_cosine_f)

    # Display the results in Streamlit
    st.header("KNN (Cosine) - With PCA - Default Parameters")
    st.subheader("Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Default Parameter [metric=Cosine] -TEST

    # Initialize an knn with a cosine Metric
    knn_test_cosine_f = KNeighborsClassifier(n_neighbors=99, metric='cosine') #sqrt

    # Measure testing time
    start_time = time.time()
    knn_test_cosine_f.fit(X_test_pca, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_cosine_f = knn_test_cosine_f.predict(X_test_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_cosine_f)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_cosine_f)

    # Display the results in Streamlit
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)


    ### Implementation of KNN (cosine) -Train Set & Test Set - With PCA. - HALVING GRID SEARCH

    # Define the parameter grid
    param_grid = {
        'n_neighbors': [87,89],
        'metric': ['cosine']
    }

    # Initialize the KNN model
    knn = KNeighborsClassifier()

    # Initialize HalvingGridSearchCV for faster results
    halving_grid_search = HalvingGridSearchCV(knn, param_grid, cv=5, factor=2,  n_jobs=-1)

    halving_grid_search.fit(X_train_pca, y_train)

    best_params = halving_grid_search.best_params_
    st.subheader("Best Hyperparameters Tuning")
    st.write(f"Best hyperparameters: {best_params}")

    #### Tuning Parameter [metric=Cosine, 'n_neighbors': 87] -TRAIN

    # Initialize an knn with a cosine Metric
    knn_train_cosine_f = KNeighborsClassifier(n_neighbors=87, metric='cosine') #sqrt

    # Measure training time
    start_time = time.time()
    knn_train_cosine_f.fit(X_train_pca, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_cosine_f = knn_train_cosine_f.predict(X_train_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_cosine_f)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_cosine_f)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Cosine, n_neighbors=87) - Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Tuning Parameter [metric=Cosine,'n_neighbors': 87] -TEST

    # Initialize an knn with a cosine Metric
    knn_test_cosine_f = KNeighborsClassifier(n_neighbors=87, metric='cosine') #sqrt

    # Measure testing time
    start_time = time.time()
    knn_test_cosine_f.fit(X_test_pca, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_cosine_f = knn_test_cosine_f.predict(X_test_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_cosine_f)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_cosine_f)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Cosine, n_neighbors=87) - Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)


    ## Implementation : Naive Bayes - Classification: GaussianBayes

    ### Implementation of NB (Gb) -Train Set and Test Set - Without Feature Selection. - DEFAULT

    #### Default Parameter GNB - TRAIN

    # Initialize an nb with a gb kernel
    nb_train_gb_wf = GaussianNB()

    # Measure training time
    start_time = time.time()
    nb_train_gb_wf.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_gb_wf = nb_train_gb_wf.predict(X_train)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_gb_wf)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_gb_wf)

    # Display the results in Streamlit
    st.header("Naive Bayes (Gaussian) - Without Feature Selection - Default Parameters")
    st.subheader("Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Default Parameter GNB - TEST

    # Measure testing time
    start_time = time.time()
    nb_train_gb_wf.fit(X_test, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_gb_wf = nb_train_gb_wf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_gb_wf)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_gb_wf)

    # Display the results in Streamlit
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)



    ### Implementation of NB (Gb) -Train Set and Test Set - Without Feature Selection. - HALVING GRID SEARCH

    # Define the parameter grid
    param_grid = {
        'var_smoothing': np.logspace(-9, 0, 10)
    }

    # Initialize the GaussianNB model
    gnb = GaussianNB()

    # Initialize HalvingGridSearchCV for faster results
    halving_grid_search = HalvingGridSearchCV(gnb, param_grid, cv=5, factor=2,  n_jobs=-1)

    halving_grid_search.fit(X_train, y_train)

    best_params = halving_grid_search.best_params_
    st.subheader("Best Hyperparameters Tuning")
    st.write(f"Best hyperparameters: {best_params}")

    #### Tuning Parameter GNB - TRAIN

    # Initialize an nb with a gb kernel
    nb_train_gb_wf = GaussianNB(var_smoothing=0.01)

    # Measure training time
    start_time = time.time()
    nb_train_gb_wf.fit(X_train, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_gb_wf = nb_train_gb_wf.predict(X_train)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_gb_wf)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_gb_wf)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Gaussian, var_smoothing=0.01) - Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Tuning Parameter GNB - TEST

    # Initialize an nb with a gb kernel
    nb_test_gb_wf = GaussianNB(var_smoothing=0.01)

    # Measure testing time
    start_time = time.time()
    nb_test_gb_wf.fit(X_test, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_gb_wf = nb_test_gb_wf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_gb_wf)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_gb_wf)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Gaussian, var_smoothing=0.01) - Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)


    ### Implementation of nb (Kernel-gb) -Train Set and Test Set- With LDA - DEFAULT

    #### Default Parameter GNB - TRAIN

    # Initialize an nb with a gb kernel
    nb_train_gb_f = GaussianNB()

    # Measure training time
    start_time = time.time()
    nb_train_gb_f.fit(X_train_lda, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_gb_f = nb_train_gb_f.predict(X_train_lda)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_gb_f)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_gb_f)

    # Display the results in Streamlit
    st.header("Naive Bayes (Gaussian) - With LDA - Default Parameters")
    st.subheader("Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Default Parameter GNB - TEST

    # Measure testing time
    start_time = time.time()
    nb_train_gb_f.fit(X_test_lda, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_gb_f = nb_train_gb_f.predict(X_test_lda)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_gb_f)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_gb_f)

    # Display the results in Streamlit
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)


    ### Implementation of nb (Kernel-gb) -Train Set and Test Set- With LDA - HALVING GRID SEARCH

    # Define the parameter grid
    param_grid = {
        'var_smoothing': np.logspace(-9, 0, 10)
    }

    # Initialize the GaussianNB model
    gnb = GaussianNB()

    # Initialize HalvingGridSearchCV for faster results
    halving_grid_search = HalvingGridSearchCV(gnb, param_grid, cv=5, factor=2,  n_jobs=-1)

    halving_grid_search.fit(X_train_lda, y_train)

    best_params = halving_grid_search.best_params_
    st.subheader("Best Hyperparameters Tuning")
    st.write(f"Best hyperparameters: {best_params}")

    #### Tuning Parameter GNB - TRAIN

    # Initialize an nb with a gb kernel
    nb_train_gb_f = GaussianNB(var_smoothing=1e-05)

    # Measure training time
    start_time = time.time()
    nb_train_gb_f.fit(X_train_lda, y_train)
    training_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_train_pred_gb_f = nb_train_gb_f.predict(X_train_lda)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    train_accuracy = accuracy_score(y_train, y_train_pred_gb_f)

    # Classification report
    class_report = classification_report(y_train, y_train_pred_gb_f)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Gaussian, var_smoothing=1e-05) - Training Results")
    st.write(f"Training time: {training_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Training Accuracy: {train_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)

    #### Tuning Parameter GNB - TEST

    # Initialize an nb with a gb kernel
    nb_test_gb_f = GaussianNB(var_smoothing=1e-05)

    # Measure testing time
    start_time = time.time()
    nb_test_gb_f.fit(X_test_lda, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred_gb_f = nb_test_gb_f.predict(X_test_lda)
    prediction_time = time.time() - start_time

    # Calculate accuracy
    test_accuracy = accuracy_score(y_test, y_test_pred_gb_f)

    # Classification report
    class_report = classification_report(y_test, y_test_pred_gb_f)

    # Display the results in Streamlit
    st.subheader("Tuned Parameters (Gaussian, var_smoothing=1e-05) - Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(class_report)




    # Initialize the models with better hyperparameters
    svm_rbf_clf = SVC(probability=True, kernel='rbf', C=10, gamma='auto')
    knn_man_clf = KNeighborsClassifier(metric='manhattan', n_neighbors=87, weights='distance', algorithm='auto')
    nb_clf = GaussianNB(var_smoothing=1e-05)

    # Create an ensemble using VotingClassifier with unique names
    ensemble_clf = VotingClassifier(estimators=[
        ('svm_rbf', svm_rbf_clf),
        ('knn_manhattan', knn_man_clf),
        ('naive_bayes', nb_clf)
    ], voting='soft', n_jobs=-1)  # Use all available cores

    # Measure testing time
    start_time = time.time()
    ensemble_clf.fit(X_test, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred = ensemble_clf.predict(X_test)
    prediction_time = time.time() - start_time

    # Calculate accuracy on test data
    test_accuracy = accuracy_score(y_test, y_test_pred)
    report = classification_report(y_test, y_test_pred)

    # Display the results in Streamlit
    st.header("Ensemble Model - Without Feature Selection")
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(report)

    ## Ensemble - SVM, KNN, NB - With PCA

    # Initialize the models with better hyperparameters
    svm_rbf_clf = SVC(probability=True, kernel='rbf', C=10, gamma='auto')
    knn_man_clf = KNeighborsClassifier(metric='manhattan', n_neighbors=87, weights='distance', algorithm='auto')
    nb_clf = GaussianNB(var_smoothing=1e-05)


    # Create an ensemble using VotingClassifier with unique names
    ensemble_clf = VotingClassifier(estimators=[
        ('svm_rbf', svm_rbf_clf),
        ('knn_manhattan', knn_man_clf),
        ('naive_bayes', nb_clf)
    ], voting='soft', n_jobs=-1)  # Use all available cores

    # Measure testing time
    start_time = time.time()
    ensemble_clf.fit(X_test_pca, y_test)
    testing_time = time.time() - start_time

    # Measure prediction time
    start_time = time.time()
    y_test_pred = ensemble_clf.predict(X_test_pca)
    prediction_time = time.time() - start_time

    # Calculate accuracy on test data
    test_accuracy = accuracy_score(y_test, y_test_pred)
    report = classification_report(y_test, y_test_pred)

    # Display the results in Streamlit
    st.header("Ensemble Model - With PCA")
    st.subheader("Testing Results")
    st.write(f"Testing time: {testing_time:.4f} seconds")
    st.write(f"Prediction time: {prediction_time:.4f} seconds")
    st.write(f"Testing Accuracy: {test_accuracy:.4f}")
    st.text("Classification Report:")
    st.text(report)

    # Define the summary results for all models
    data = {
        'Model': [
            'SVM (Linear) - Default', 'SVM (Linear) - Tuned',
            'SVM (Linear + PCA) - Default', 'SVM (Linear + PCA) - Tuned',
            'SVM (Poly) - Default', 'SVM (Poly) - Tuned',
            'SVM (Poly + PCA) - Default', 'SVM (Poly + PCA) - Tuned',
            'SVM (RBF) - Default', 'SVM (RBF) - Tuned',
            'SVM (RBF + PCA) - Default', 'SVM (RBF + PCA) - Tuned',
            'KNN (Euclidean) - Default', 'KNN (Euclidean) - Tuned',
            'KNN (Euclidean + PCA) - Default', 'KNN (Euclidean + PCA) - Tuned',
            'KNN (Manhattan) - Default', 'KNN (Manhattan) - Tuned',
            'KNN (Manhattan + PCA) - Default', 'KNN (Manhattan + PCA) - Tuned',
            'KNN (Cosine) - Default', 'KNN (Cosine) - Tuned',
            'KNN (Cosine + PCA) - Default', 'KNN (Cosine + PCA) - Tuned',
            'NB (Gaussian) - Default', 'NB (Gaussian) - Tuned',
            'NB (Gaussian + LDA) - Default', 'NB (Gaussian + LDA) - Tuned',
            'Ensemble - Without Feature Selection', 'Ensemble - With PCA'
        ],
        'Training Accuracy': [
            0.8455, 0.8658, 0.8458, 0.8675, 0.8206, 0.8757, 0.8207, 0.8840, 0.9054, 0.9513, 0.9055, 0.9579,
            0.9405, 0.9455, 0.9405, 0.9455, 0.9510, 0.9563, 0.9446, 0.9467, 0.9383, 0.9442, 0.9368, 0.9440,
            0.7887, 0.7876, 0.8093, 0.8093, 0.9614, 0.9811
        ],
        'Testing Accuracy': [
            0.8775, 0.8864, 0.8764, 0.8814, 0.8014, 0.8652, 0.8000, 0.8661, 0.9005, 0.9427, 0.9007, 0.9596,
            0.9330, 0.9313, 0.9330, 0.9313, 0.9402, 0.9389, 0.9339, 0.9339, 0.9257, 0.9268, 0.9257, 0.9270,
            0.7836, 0.7837, 0.8018, 0.8018, 0.9614, 0.9811
        ]
    }

    # Create the DataFrame
    summary_df = pd.DataFrame(data)

    # Display the summary table in Streamlit
    st.header("Summary of Model Results (60:40 Split Ratios)")
    st.dataframe(summary_df)


    # Visualization
    st.header("Visualization of Model Performance")

    ## Average Performance of Different Models
    svm_avg_accuracy = 0.8768
    knn_avg_accuracy = 0.9315
    nb_avg_accuracy = 0.7948
    ensemble_avg_accuracy = 0.9709

    labels = ['SVM', 'KNN', 'NB', 'Ensemble']
    avg_accuracies = [svm_avg_accuracy, knn_avg_accuracy, nb_avg_accuracy, ensemble_avg_accuracy]
    colors = ['blue', 'green', 'red', 'purple']

    fig, ax = plt.subplots()
    bars = ax.bar(labels, avg_accuracies, color=colors, alpha=0.7)
    ax.set_xlabel('Model', fontsize=8)
    ax.set_ylabel('Average Test Accuracy', fontsize=8)
    ax.set_title('Average Performance of Different Models', fontsize=8)

    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 4), ha='center', va='bottom',fontsize=8)

    st.pyplot(fig)

    ## Average Performance Across All Combinations
    accuracy_values = {
        'Linear SVM': [0.874, 0.8733, 0.8775, 0.8764, 0.8886, 0.8864, 0.8864, 0.8814],
        'Polynomial SVM': [0.7988, 0.7986, 0.8014, 0.8, 0.8529, 0.8629, 0.8652, 0.8661],
        'RBF SVM': [0.8881, 0.8881, 0.9005, 0.9007, 0.926, 0.9467, 0.9427, 0.9596],
        'KNN Euclidean': [0.9348, 0.9355, 0.935, 0.9357, 0.933, 0.933, 0.9313, 0.9313],
        'KNN Manhattan': [0.9417, 0.9321, 0.9412, 0.9333, 0.9402, 0.9339, 0.9389, 0.9339],
        'KNN Cosine': [0.9207, 0.9212, 0.9257, 0.9261, 0.9221, 0.9221, 0.9268, 0.927],
        'Naive Bayes': [0.7886, 0.8019, 0.7836, 0.8018, 0.795, 0.8019, 0.7837, 0.8018],
        'Ensemble': [0.9574, 0.9829, 0.9623, 0.9811]
    }

    average_accuracies = {model: np.mean(acc) for model, acc in accuracy_values.items()}
    sorted_accuracies = dict(sorted(average_accuracies.items(), key=lambda item: item[1]))

    fig, ax = plt.subplots(figsize=(13, 10))
    colors = plt.cm.viridis(np.linspace(0, 1, len(sorted_accuracies)))
    bars = ax.bar(sorted_accuracies.keys(), sorted_accuracies.values(), color=colors, edgecolor='black')

    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, yval + 0.01, round(yval, 4), ha='center', va='bottom', fontsize=10)

    ax.set_xlabel('Model', fontsize=12)
    ax.set_ylabel('Average Test Accuracy', fontsize=12)
    ax.set_title('Average Performance Across All Combinations', fontsize=16)
    ax.set_ylim([0, 1])
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.axhline(y=0.5, color='r', linestyle='--', linewidth=1)

    plt.xticks(fontsize=10)
    plt.tight_layout()
    st.pyplot(fig)

    ## Impact of Different Kernels in SVM
    data = {
        'Kernel': ['Linear', 'Polynomial', 'RBF'],
        'Default_70_30': [0.874, 0.7988, 0.8881],
        'Tuned_70_30': [0.8886, 0.8529, 0.926],
        'Default_60_40': [0.8775, 0.8014, 0.9005],
        'Tuned_60_40': [0.8864, 0.8652, 0.9427],
        'Default_70_30_FS': [0.8733, 0.7986, 0.8881],
        'Tuned_70_30_FS': [0.8864, 0.8629, 0.9467],
        'Default_60_40_FS': [0.8764, 0.8000, 0.9007],
        'Tuned_60_40_FS': [0.8814, 0.8661, 0.9596]
    }

    df = pd.DataFrame(data)
    df['Average'] = df[['Default_70_30', 'Tuned_70_30', 'Default_60_40', 'Tuned_60_40',
                        'Default_70_30_FS', 'Tuned_70_30_FS', 'Default_60_40_FS', 'Tuned_60_40_FS']].mean(axis=1)

    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(df['Kernel'], df['Average'], color=['red', 'blue', 'green'])
    ax.set_xlabel('SVM Kernel')
    ax.set_ylabel('Average Test Accuracy')
    ax.set_title('Comparison of SVM Kernels (Average Test Accuracy)')
    ax.set_ylim(0, 1)
    ax.grid(axis='y')

    for index, value in enumerate(df['Average']):
        ax.text(index, value + 0.01, f'{value:.4f}', ha='center', fontweight='bold')

    st.pyplot(fig)

    ## Impact of Different Metrics in KNN
    data = {
        'Kernel': ['Euclidean', 'Manhattan', 'Cosine'],
        'Default_70_30': [0.9348, 0.9417, 0.9207],
        'Tuned_70_30': [0.9355, 0.9412, 0.9221],
        'Default_60_40': [0.9330, 0.9402, 0.9257],
        'Tuned_60_40': [0.9313, 0.9389, 0.9268],
        'Default_70_30_FS': [0.9350, 0.9321, 0.9212],
        'Tuned_70_30_FS': [0.9357, 0.9333, 0.9221],
        'Default_60_40_FS': [0.9330, 0.9339, 0.9261],
        'Tuned_60_40_FS': [0.9313, 0.9339, 0.9270]
    }

    df = pd.DataFrame(data)
    df['Average'] = df[['Default_70_30', 'Tuned_70_30', 'Default_60_40', 'Tuned_60_40',
                        'Default_70_30_FS', 'Tuned_70_30_FS', 'Default_60_40_FS', 'Tuned_60_40_FS']].mean(axis=1)

    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(df['Kernel'], df['Average'], color=['red', 'blue', 'green'])
    ax.set_xlabel('KNN Metrics')
    ax.set_ylabel('Average Test Accuracy')
    ax.set_title('Comparison of KNN Metrics (Average Test Accuracy)')
    ax.set_ylim(0, 1)
    ax.grid(axis='y')

    for index, value in enumerate(df['Average']):
        ax.text(index, value + 0.01, f'{value:.4f}', ha='center', fontweight='bold')

    st.pyplot(fig)

    ## Impact of Feature Selection on Model Performance
    accuracy_70_30 = {
        "without_fs": [0.874, 0.8886, 0.7988, 0.8529, 0.8881, 0.926, 0.9348, 0.9355, 0.9417, 0.9412, 0.9207, 0.9221, 0.7886, 0.795, 0.9574],
        "with_fs": [0.8733, 0.8864, 0.7986, 0.8629, 0.8881, 0.9467, 0.935, 0.9357, 0.9321, 0.9333, 0.9212, 0.9221, 0.8019, 0.8019, 0.9829]
    }

    accuracy_60_40 = {
        "without_fs": [0.8775, 0.8864, 0.8014, 0.8652, 0.9005, 0.9427, 0.933, 0.9313, 0.9402, 0.9389, 0.9257, 0.9268, 0.7836, 0.7837, 0.9623],
        "with_fs": [0.8764, 0.8814, 0.8, 0.8661, 0.9007, 0.9596, 0.933, 0.9313, 0.9339, 0.9339, 0.9261, 0.927, 0.8018, 0.8018, 0.9811]
    }

    average_without_fs = np.mean(accuracy_70_30["without_fs"] + accuracy_60_40["without_fs"])
    average_with_fs = np.mean(accuracy_70_30["with_fs"] + accuracy_60_40["with_fs"])

    categories = ['Without Feature Selection', 'With Feature Selection']
    averages = [average_without_fs, average_with_fs]

    fig, ax = plt.subplots(figsize=(6, 4))
    bars = ax.bar(categories, averages, color=['blue', 'green'], width=0.8)

    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2.0, height, f'{height:.4f}', ha='center', va='bottom')

    ax.set_xlabel('Feature Selection', fontsize=8)
    ax.set_ylabel('Average Test Accuracy', fontsize=8)
    ax.set_title('Comparison of Test Accuracy with and without Feature Selection', fontsize=8)
    ax.set_ylim(0, 1)

    st.pyplot(fig)

    ## Default vs Tuned Models
    data = {
        'Model': ['SVM_Linear', 'SVM_Polynomial', 'SVM_RBF', 'KNN_Euclidean', 'KNN_Manhattan', 'KNN_Cosine', 'NB_Gaussian'],
        'Default_70_30': [0.874, 0.7988, 0.8881, 0.9348, 0.9417, 0.9207, 0.7886],
        'Default_60_40': [0.8775, 0.8014, 0.9005, 0.933, 0.9402, 0.9257, 0.7836],
        'Tuned_70_30': [0.8886, 0.8529, 0.926, 0.9355, 0.9412, 0.9221, 0.795],
        'Tuned_60_40': [0.8864, 0.8652, 0.9427, 0.9313, 0.9389, 0.9268, 0.7837],
        'Default_FS_70_30': [0.8733, 0.7986, 0.8881, 0.935, 0.9321, 0.9212, 0.8019],
        'Default_FS_60_40': [0.8764, 0.8, 0.9007, 0.933, 0.9339, 0.9261, 0.8018],
        'Tuned_FS_70_30': [0.8884, 0.8629, 0.9467, 0.9357, 0.9333, 0.9221, 0.8019],
        'Tuned_FS_60_40': [0.8814, 0.8661, 0.9596, 0.9313, 0.9339, 0.927, 0.8018]
    }

    df = pd.DataFrame(data)
    df['Default_Avg'] = df[['Default_70_30', 'Default_60_40']].mean(axis=1)
    df['Tuned_Avg'] = df[['Tuned_70_30', 'Tuned_60_40']].mean(axis=1)
    df['Default_FS_Avg'] = df[['Default_FS_70_30', 'Default_FS_60_40']].mean(axis=1)
    df['Tuned_FS_Avg'] = df[['Tuned_FS_70_30', 'Tuned_FS_60_40']].mean(axis=1)

    default_avg = df['Default_Avg'].mean()
    tuned_avg = df['Tuned_Avg'].mean()
    default_fs_avg = df['Default_FS_Avg'].mean()
    tuned_fs_avg = df['Tuned_FS_Avg'].mean()

    labels_without_fs = ['Default', 'Tuned']
    averages_without_fs = [default_avg, tuned_avg]
    colors_without_fs = ['#1f77b4', '#ff7f0e']

    x_without_fs = np.arange(len(labels_without_fs))
    width = 0.4

    fig1, ax1 = plt.subplots(figsize=(10, 6))
    rects1 = ax1.bar(x_without_fs, averages_without_fs, width, label='Accuracy', color=colors_without_fs, edgecolor='black')

    ax1.set_ylabel('Average Accuracy', fontsize=14)
    ax1.set_title('Average Accuracy by Default and Tuned Models without Feature Selection', fontsize=16)
    ax1.set_xticks(x_without_fs)
    ax1.set_xticklabels(labels_without_fs, fontsize=12)
    ax1.grid(True, axis='y', linestyle='--', linewidth=0.7)

    for rect in rects1:
        height = rect.get_height()
        ax1.annotate('{}'.format(round(height, 4)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 5),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=12, fontweight='bold')

    fig1.tight_layout()
    st.pyplot(fig1)

    labels_with_fs = ['Default_FS', 'Tuned_FS']
    averages_with_fs = [default_fs_avg, tuned_fs_avg]
    colors_with_fs = ['#2ca02c', '#d62728']

    x_with_fs = np.arange(len(labels_with_fs))
    width = 0.4

    fig2, ax2 = plt.subplots(figsize=(10, 6))
    rects2 = ax2.bar(x_with_fs, averages_with_fs, width, label='Accuracy', color=colors_with_fs, edgecolor='black')

    ax2.set_ylabel('Average Accuracy', fontsize=14)
    ax2.set_title('Average Accuracy by Default and Tuned Models with Feature Selection', fontsize=16)
    ax2.set_xticks(x_with_fs)
    ax2.set_xticklabels(labels_with_fs, fontsize=12)
    ax2.grid(True, axis='y', linestyle='--', linewidth=0.7)

    for rect in rects2:
        height = rect.get_height()
        ax2.annotate('{}'.format(round(height, 4)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 5),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=12, fontweight='bold')

    fig2.tight_layout()
    st.pyplot(fig2)

    ## Performance Across Different Splitting Ratios
    data_split = {
        'Model': ['SVM_Linear', 'SVM_Polynomial', 'SVM_RBF', 'KNN_Euclidean', 'KNN_Manhattan', 'KNN_Cosine', 'NB_Gaussian'],
        'Default_70_30': [0.874, 0.7988, 0.8881, 0.9348, 0.9417, 0.9207, 0.7886],
        'Default_60_40': [0.8775, 0.8014, 0.9005, 0.933, 0.9402, 0.9257, 0.7836],
        'Tuned_70_30': [0.8886, 0.8529, 0.926, 0.9355, 0.9412, 0.9221, 0.795],
        'Tuned_60_40': [0.8864, 0.8652, 0.9427, 0.9313, 0.9389, 0.9268, 0.7837],
        'Default_FS_70_30': [0.8733, 0.7986, 0.8881, 0.935, 0.9321, 0.9212, 0.8019],
        'Default_FS_60_40': [0.8764, 0.8, 0.9007, 0.933, 0.9339, 0.9261, 0.8018],
        'Tuned_FS_70_30': [0.8884, 0.8629, 0.9467, 0.9357, 0.9333, 0.9221, 0.8019],
        'Tuned_FS_60_40': [0.8814, 0.8661, 0.9596, 0.9313, 0.9339, 0.927, 0.8018]
    }

    df_split = pd.DataFrame(data_split)
    df_split['Avg_70_30'] = df_split[['Default_70_30', 'Tuned_70_30']].mean(axis=1)
    df_split['Avg_60_40'] = df_split[['Default_60_40', 'Tuned_60_40']].mean(axis=1)
    df_split['Avg_FS_70_30'] = df_split[['Default_FS_70_30', 'Tuned_FS_70_30']].mean(axis=1)
    df_split['Avg_FS_60_40'] = df_split[['Default_FS_60_40', 'Tuned_FS_60_40']].mean(axis=1)

    avg_70_30 = df_split['Avg_70_30'].mean()
    avg_60_40 = df_split['Avg_60_40'].mean()
    avg_fs_70_30 = df_split['Avg_FS_70_30'].mean()
    avg_fs_60_40 = df_split['Avg_FS_60_40'].mean()

    labels_without_fs = ['70/30 Split', '60/40 Split']
    averages_without_fs = [avg_70_30, avg_60_40]
    colors_without_fs = ['#1f77b4', '#ff7f0e']

    x_without_fs = np.arange(len(labels_without_fs))
    width = 0.4

    fig1, ax1 = plt.subplots(figsize=(10, 6))
    rects1 = ax1.bar(x_without_fs, averages_without_fs, width, label='Accuracy', color=colors_without_fs, edgecolor='black')

    ax1.set_ylabel('Average Accuracy', fontsize=14)
    ax1.set_title('Average Accuracy by Splitting Ratio without Feature Selection', fontsize=16)
    ax1.set_xticks(x_without_fs)
    ax1.set_xticklabels(labels_without_fs, fontsize=12)
    ax1.grid(True, axis='y', linestyle='--', linewidth=0.7)

    for rect in rects1:
        height = rect.get_height()
        ax1.annotate('{}'.format(round(height, 4)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 5),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=12, fontweight='bold')

    fig1.tight_layout()
    st.pyplot(fig1)

    labels_with_fs = ['70/30 Split (FS)', '60/40 Split (FS)']
    averages_with_fs = [avg_fs_70_30, avg_fs_60_40]
    colors_with_fs = ['#2ca02c', '#d62728']

    x_with_fs = np.arange(len(labels_with_fs))
    width = 0.4

    fig2, ax2 = plt.subplots(figsize=(10, 6))
    rects2 = ax2.bar(x_with_fs, averages_with_fs, width, label='Accuracy', color=colors_with_fs, edgecolor='black')

    ax2.set_ylabel('Average Accuracy', fontsize=14)
    ax2.set_title('Average Accuracy by Splitting Ratio with Feature Selection', fontsize=16)
    ax2.set_xticks(x_with_fs)
    ax2.set_xticklabels(labels_with_fs, fontsize=12)
    ax2.grid(True, axis='y', linestyle='--', linewidth=0.7)

    for rect in rects2:
        height = rect.get_height()
        ax2.annotate('{}'.format(round(height, 4)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 5),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=12, fontweight='bold')

    fig2.tight_layout()
    st.pyplot(fig2)


else:
      st.write("Please upload a CSV file to begin.")

Writing Ddos_Attack_Detection2.py


In [ ]:
!streamlit run Ddos_Attack_Detection2.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.13.119:8501

npx: installed 22 in 4.291s
your url is: https://early-poems-smile.loca.lt
/content/Ddos_Attack_Detection2.py:28: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([chunk for chunk in pd.read_csv(uploaded_file, chunksize=chunk_size)])
/content/Ddos_Attack_Detection2.py:28: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat([chunk for chunk in pd.read_csv(uploaded_file, chunksize=chunk_size)])
/content/Ddos_Attack_Detection2.py:173: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.countplot(x=' Label', data=df, palette='coolwarm')
/content/